In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

## Preparing a list of URLs to scrape

In [2]:
root_url = 'https://www.zomato.com/kolkata/dinner'

In [3]:
# Lying to the server so that we can scrape without consequences
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

In [4]:
rest_urls = []
for i in range(1, 151):
    # Downloading page containing list of restaurants
    page = requests.get(f'{root_url}?page={i}', headers=headers)
    
    # Cooking a soup for easy digestion
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Extracting a list of all restaurants
    res = soup.find_all('div', attrs={'class': 'search_left_featured clearfix'})
    
    for entry in res:
        # Only storing the URL of the restaurant
        url = entry.find('a', href=True)['href']
        
        # Replacing /info with /reviews so as to navigate to the reviews directly
        rest_urls.append(url.replace('/info', '/reviews'))
    print(f'Page [{i}] scraped!')

Page [1] scraped!
Page [2] scraped!
Page [3] scraped!
Page [4] scraped!
Page [5] scraped!
Page [6] scraped!
Page [7] scraped!
Page [8] scraped!
Page [9] scraped!
Page [10] scraped!
Page [11] scraped!
Page [12] scraped!
Page [13] scraped!
Page [14] scraped!
Page [15] scraped!
Page [16] scraped!
Page [17] scraped!
Page [18] scraped!
Page [19] scraped!
Page [20] scraped!
Page [21] scraped!
Page [22] scraped!
Page [23] scraped!
Page [24] scraped!
Page [25] scraped!
Page [26] scraped!
Page [27] scraped!
Page [28] scraped!
Page [29] scraped!
Page [30] scraped!
Page [31] scraped!
Page [32] scraped!
Page [33] scraped!
Page [34] scraped!
Page [35] scraped!
Page [36] scraped!
Page [37] scraped!
Page [38] scraped!
Page [39] scraped!
Page [40] scraped!
Page [41] scraped!
Page [42] scraped!
Page [43] scraped!
Page [44] scraped!
Page [45] scraped!
Page [46] scraped!
Page [47] scraped!
Page [48] scraped!
Page [49] scraped!
Page [50] scraped!
Page [51] scraped!
Page [52] scraped!
Page [53] scraped!
Pa

In [5]:
len(rest_urls)

2235

## Scraping reviews and corresponding ratings

In [6]:
texts = []
labels = []
pbar = tqdm(rest_urls)
for url in pbar:
    try:
        # A little verbosity doesn't hurt
        pbar.set_description(f'Fetching {url}')
        
        # Downloading the restaurant page
        page = requests.get(url, headers=headers)
        
        # Cooking a soup
        page = BeautifulSoup(page.text, 'html.parser')
        
        # Looking for reviews
        reviews = page.find_all('div', attrs={'class': 'rev-text mbot0 '})
        
        for review in reviews:
            # Extracting the label
            label = float(review.find('div')['aria-label'][-3:])
            
            # Clearing out some junk
            review.find('div').decompose()
            
            # Extracting the text
            text = review.text.strip()

            # Storing our treasures
            labels.append(label)
            texts.append(text)
            
    except KeyboardInterrupt:
        # Manually breaking the loop
        break   
    except:
        # Something went wrong so ...
        # Skipping the current url :)
        print(f'Failed to retrieve url: {url}')

# Making a pandas DataFrame from our data
data = pd.DataFrame()
data['text'] = texts
data['label'] = labels

Fetching https://www.zomato.com/kolkata/calcutta-64-sector-1-salt-lake/reviews:   4%|▍         | 84/2235 [01:35<40:51,  1.14s/it]                       

Failed to retrieve url: https://www.zomato.com/kolkata/tung-fong-park-street-area/reviews


Fetching https://www.zomato.com/kolkata/hotel-priya-shyam-bazar/reviews: 100%|██████████| 2235/2235 [34:10<00:00,  1.09it/s]                                            


## Glimpse of the data

In [7]:
data.head()

,text,label
0,One thing I loved about the place is the view ...,4.0
1,"If You are going to Ozora, Please Select Rooft...",4.0
2,What a wonderful cold winter evening it was. M...,4.0
3,Well had been to this place a while back to at...,3.0
4,Visited on a cool evening and was mesmerized b...,4.0


In [8]:
data.shape

(11357, 2)

## Saving the data to disk

In [10]:
data.to_csv('data/reviews.csv', index=False)